# Raw to CSV

Process the raw CSV files from the I-V Curve Tracer to get 1 CSV file with only the relevant columns.

In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
import time
from datetime import datetime 

In [2]:
base_path = Path(r"D:\Research\Data") # Base path for raw data files
output_csv = Path("unfiltered_dataset.csv") # Output CSV file path
log_file = Path("processed_files.txt") # Log file to keep track of processed files
error_log = Path("error_log.txt") # Log file for errors
missing_csvs = Path("missing_csv.txt") # Log missing csv filenames
print_every = 2000 # Print progress every 2000 files

### Identified columns on CSV

In [3]:
row1_column_names = [
    'date', 'time_start', 'voltage_0', 'current_0', 'power_0', 'wavelength_0', 'spectralirr_0',
    'modtemp_c', 'modtemp_l', 'cell_v', 'irr_horiz_start', 'irr_incl20_start', 'airtemp',
    'humidity_rel', 'pressure_rel', 'air_density', 'wind_speed_kmh',
    'wind_dir', 'humidity_abs', 'pressure_abs', 'wind_speed_ms', 'irr_east_start',
    'irr_west_start', 'irr_floor_ref_start', 'irr_diffuse_start', 'irr_incl15_start'
]

row2_column_names = [
    'module_name', 'time_end', 'voltage_1', 'current_1',
    'power_1', 'wavelength_1', 'spectralirr_1', 'cell_v_end',
    'irr_horiz_end', 'irr_incl20_end', 'irr_east_end', 'irr_west_end',
    'irr_floor_ref_end', 'irr_diffuse_end', 'irr_incl15_end'
]

### Error log

This will log errors into a file with the corresponding exception.

In [4]:
def log_error(filename, message, exception=None):
    if exception:
        full_msg = f"{filename} — {message}: {repr(exception)}"
    else:
        full_msg = f"{filename} — {message}"

    # Print to console
    print("❌", full_msg)

    # Append to log file
    with open(error_log, "a", encoding="utf-8") as err_f:
        err_f.write(full_msg + "\n")

# Processing functions

In [5]:
def process_file(file_path):
    filename = file_path.name

    # Read CSV
    try: df = pd.read_csv(file_path, sep=';', header=None) 
    except Exception as e:
        log_error(filename, "Error reading CSV", e)
        return None

    # Parse the first 2 columns using the column names defined before.
    try:
        df_row1 = pd.DataFrame([df.iloc[0, :].dropna().tolist()], columns=row1_column_names)
        df_row2 = pd.DataFrame([df.iloc[1, :].dropna().tolist()], columns=row2_column_names)
        # Convert to Series for easier access
        row1 = df_row1.iloc[0]
        row2 = df_row2.iloc[0]
    except Exception as e:
        log_error(filename, "Error parsing metadata rows", e)
        return None

    try:
        # Generate timestamp from filename
        name = filename.replace('.csv', '')
        # Split by underscore
        parts = name.split('_')
        # Extract date/time parts (skip module name)
        day = int(parts[1])
        month = int(parts[2])
        year = int(parts[3])
        hour = int(parts[4])
        minute = int(parts[5])
        second = int(parts[6])
        
        timestamp = datetime(year, month, day, hour, minute, second)
        
        module_name = row2["module_name"]
    except Exception as e:
        log_error(filename, "Missing metadata fields", e)
        return None
    
    # Compute Spectral Integral
    try:
        wavelength = df.iloc[:2048, 5].copy() # Columns 5 is wavelength (nm) 
        spectral_irradiance = df.iloc[:2048, 6].copy() # Column 6 is spectral irradiance (W/m2/nm)
        computed_irradiance = np.trapezoid(spectral_irradiance, x=wavelength)
    except Exception as e:
        log_error(filename, "Error computing spectral integral", e)
        return None

    # Get Irradiances
    try:
        # Start irradiances (from row1)
        irr_horiz_start = row1["irr_horiz_start"]
        irr_incl20_start = row1["irr_incl20_start"]
        irr_incl15_start = row1["irr_incl15_start"]
        irr_east_start = row1["irr_east_start"]
        irr_west_start = row1["irr_west_start"]
        irr_floor_ref_start = row1["irr_floor_ref_start"]
        
        # End irradiances (from row2)
        irr_horiz_end = row2["irr_horiz_end"]
        irr_incl20_end = row2["irr_incl20_end"]
        irr_incl15_end = row2["irr_incl15_end"]
        irr_east_end = row2["irr_east_end"]
        irr_west_end = row2["irr_west_end"]
        irr_floor_ref_end = row2["irr_floor_ref_end"]
    except Exception as e:
        log_error(filename, "Missing irradiance data", e)
        return None

    # Get temperature
    try:
        modtemp_c = row1["modtemp_c"]
        modtemp_l = row1["modtemp_l"]
    except Exception as e:
        log_error(filename, "Error reading module temperatures", e)
        return None

    # I-V & P-V Curve validation
    try: 
        measurements = df.iloc[:, [2, 3, 4]].copy()
        measurements.columns = ['voltage', 'current', 'power']
        # Discard the first values (start from voltage close to 0)
        min_idx = measurements['voltage'].idxmin()
        measurements = measurements.iloc[min_idx:].reset_index(drop=True)

    except Exception as e:
        log_error(filename, "Error processing I-V curve", e)
        return None

    try:
        V = measurements["voltage"]
        I = measurements["current"]
        P = measurements["power"]

        Voc = V.max()
        Pmpp = P.max()
        mpp_idx = P.idxmax()
        Vmpp = V[mpp_idx]
        Impp = I[mpp_idx]
        Imin = I.min()
        
    except Exception as e:
        log_error(filename, "Error computing electrical parameters", e)
        return None

    # Delete tail
    try:
        d = np.mean(np.abs(np.diff(I[:200])))
        diffs_20 = np.abs(np.diff(I[:20]))
        # Compare diffs20 to d
        idxs_cola = [i for i, diff in enumerate(diffs_20) if diff > d]
        if idxs_cola:
            corte_inicial = idxs_cola[-1] + 1
            measurements = measurements.iloc[corte_inicial:].reset_index(drop=True)
    except Exception as e:
        log_error(filename, "Error cleaning up cola noise", e)

    # Find Isc
    try:
        ordered_meas = measurements.sort_values(by="voltage").reset_index(drop=True)
        V = ordered_meas["voltage"].values  # Convert to numpy array
        I = ordered_meas["current"].values  # Convert to numpy array

        
        # 1) Exact V = 0 present → take mean current at those points
        zero_mask = np.isclose(V, 0.0)
        if zero_mask.any():
            Isc = float(np.mean(I[zero_mask]))
        
        # 2) No exact V = 0, but V crosses zero → linear interpolation
        elif (V.min() < 0) and (V.max() > 0):
            found = False
            for i in range(len(V) - 1):
                if (V[i] < 0) and (V[i + 1] > 0):
                    dv = V[i + 1] - V[i]
                    if dv == 0:
                        log_error(filename, "Duplicate voltages at zero crossing", None)
                        Isc = float(I[i])  # Fallback: use current at that point
                    else:
                        t = (0.0 - V[i]) / dv
                        Isc = float(I[i] + t * (I[i + 1] - I[i]))
                    found = True
                    break
            
            if not found:
                log_error(filename, "Could not find Isc crossing despite voltage sign change", None)
                return None
        
        # 3) No zero crossing, use the two closest points to zero for extrapolation
        else:
            if len(V) < 2:
                log_error(filename, "Insufficient data points for Isc extrapolation", None)
                return None
            
            v1, v2 = V[0], V[1]
            i1, i2 = I[0], I[1]
            
            dv = v2 - v1
            if dv == 0:
                log_error(filename, "Duplicate voltages in extrapolation", None)
                return None
            
            m = (i2 - i1) / dv
            Isc = float(i1 - m * v1)
            
    except Exception as e:
        log_error(filename, "Error estimating Isc", e)
        return None
    
    # Build final dataframe
    try:
        final_df = pd.DataFrame({
            # Metadata
            'filename': [filename],
            'module_name': [module_name],
            'timestamp': [timestamp],

            # MPP
            'Vmpp': [Vmpp],
            'Impp': [Impp],
            'Pmpp': [Pmpp],

            # Electrical characteristics
            'Voc': [Voc],
            'Imin': [Imin],
            'Isc': [Isc],

            # Irradiance
            'G_spec_int': [computed_irradiance],
            'G_tilt20_start': [irr_incl20_start],
            'G_tilt15_start': [irr_incl15_start],
            'G_horiz_start': [irr_horiz_start],
            'G_east_start': [irr_east_start],
            'G_west_start': [irr_west_start],
            'G_refl_start': [irr_floor_ref_start],

            'G_horiz_end': [irr_horiz_end],
            'G_tilt20_end': [irr_incl20_end],
            'G_tilt15_end': [irr_incl15_end],
            'G_east_end': [irr_east_end],
            'G_west_end': [irr_west_end],
            'G_refl_end': [irr_floor_ref_end],

            # Environmental characteristics
            'module_temperature_center': [modtemp_c],
            'module_temperature_lateral': [modtemp_l],
            'air_temperature': [row1['airtemp']],
            'relative_humidity': [row1['humidity_rel']],
            'air_density': [row1['air_density']],
            'abs_pressure': [row1['pressure_abs']],
            'wind_speed_ms': [row1['wind_speed_ms']],
            'wind_direction': [row1['wind_dir']]
        })
    except Exception as e:
        log_error(filename, "Error creating final dataframe", e)
        return None

    return final_df

In [ ]:
# Get all CSV files from 2023-2025
all_files = []
skip_prefixes = ('MS711', 'WS500', 'PV')

for year in range(2023, 2026):  # 2023, 2024, 2025
    year_files = list(base_path.glob(f"{year}/**/*.csv"))
    # Filter out unwanted prefixes
    year_files = [f for f in year_files if not f.name.startswith(skip_prefixes)]
    all_files.extend(year_files)
    print(f"Found {len(year_files)} files in {year}")

print(f"\nTotal: {len(all_files)} CSV files from 2023-2025 (excluding MS711, WS500, PV)")

# Load already processed files
processed_files = set()
if log_file.exists():
    with open(log_file, 'r') as f:
        processed_files = set(line.strip() for line in f)
    print(f"Already processed: {len(processed_files)} files")

# Filter to only new files
files_to_process = [f for f in all_files if f.name not in processed_files]
total_files = len(files_to_process)

print(f"Files to process: {total_files}")

# Determine if we need header
write_header = not output_csv.exists() or output_csv.stat().st_size == 0

Found 426187 files in 2023
Found 441706 files in 2024
Found 425818 files in 2025

Total: 1293711 CSV files from 2023-2025
Files to process: 1293711


# Process Loop

In [18]:
processed_count = 0
skipped_due_to_error = 0
start_time = time.time()

with open(output_csv, 'a', newline='', encoding='utf-8') as output_f:
    for i, file_path in enumerate(files_to_process):  # ✅ Use files_to_process
        filename = file_path.name  # ✅ Just filename for consistency
        
        try:
            result_df = process_file(file_path)
        except Exception as e:
            print(f"Unhandled error on file: {filename}")
            skipped_due_to_error += 1
            continue

        if result_df is None:
            skipped_due_to_error += 1
            continue

        result_df.to_csv(output_f, header=write_header, index=False)
        write_header = False

        with open(log_file, 'a') as log_f:
            log_f.write(filename + '\n')  

        processed_count += 1

        # Progress print
        if processed_count % print_every == 0 or processed_count == 1:
            elapsed = time.time() - start_time
            avg_time = elapsed / processed_count
            remaining = avg_time * (total_files - processed_count)
            print(
                f"[{processed_count}/{total_files}] processed "
                f"({processed_count/total_files * 100:.1f}%) "
                f"— Skipped: {skipped_due_to_error} "
                f"— ETA: {remaining / 60:.1f} min"
            )


# Final summary
print(f"\n✅ Extraction complete. Processed: {processed_count}, Skipped due to error: {skipped_due_to_error}")

[1/1293711] processed (0.0%) — Skipped: 0 — ETA: 426.7 min
❌ CS3K320MB-V_01_01_2023_12_09_22.csv — Error parsing metadata rows: ValueError(AssertionError('26 columns passed, passed data had 25 columns'))
❌ CS3K320MB-V_01_01_2023_12_14_23.csv — Error parsing metadata rows: ValueError(AssertionError('26 columns passed, passed data had 25 columns'))


C:\Users\syaba\AppData\Local\Temp\ipykernel_22988\4081861802.py:5: DtypeWarning: Columns (2,3,5,6,7,8,2057,2058,2059,2060,2061,2062,2063,2064,2065,2066,2067,2068,2069) have mixed types. Specify dtype option on import or set low_memory=False.
  try: df = pd.read_csv(file_path, sep=';', header=None)


❌ MS711_01_01_2023.csv — Error parsing metadata rows: ValueError(AssertionError('26 columns passed, passed data had 2070 columns'))
❌ PV_01_01_2023.csv — Error parsing metadata rows: ValueError(AssertionError('26 columns passed, passed data had 40 columns'))
❌ WS500_01_01_2023.csv — Error reading CSV: UnicodeDecodeError('utf-8', b'DATE;TIME;AIR TEMPERATURE (\xb0C);RELATIVE HUMIDITY (%);RELATIVE PRESSURE (hPa);AIR DENSITIY (Kg/m3);WIND SPEED (km/h);WIND DIRECTION (\xb0);ABSOLUTE HUMIDITY (%);ABSOLUTE PRESSURE (hPa);WIND SPEED (m/s)\r\n01/01/2023;00:00:00;20.12;86.88;1006.80;1.19;4.59;189.37;15.09;1006.80;1.28\r\n01/01/2023;00:01:00;20.11;87.02;1006.80;1.19;3.02;34.43;15.10;1006.80;0.84\r\n01/01/2023;00:02:00;20.11;87.26;1006.76;1.19;2.39;209.11;15.14;1006.76;0.66\r\n01/01/2023;00:03:00;20.11;87.34;1006.75;1.19;2.07;0.42;15.15;1006.75;0.57\r\n01/01/2023;00:04:00;20.09;87.63;1006.72;1.19;1.23;277.30;15.19;1006.72;0.34\r\n01/01/2023;00:05:00;20.08;87.54;1006.74;1.19;1.71;243.22;15.16;1006.

C:\Users\syaba\AppData\Local\Temp\ipykernel_22988\4081861802.py:5: DtypeWarning: Columns (2,3,5,6,7,8,2057,2058,2059,2060,2061,2062,2063,2064,2065,2066,2067,2068,2069) have mixed types. Specify dtype option on import or set low_memory=False.
  try: df = pd.read_csv(file_path, sep=';', header=None)


❌ MS711_02_01_2023.csv — Error parsing metadata rows: ValueError(AssertionError('26 columns passed, passed data had 2070 columns'))
❌ PV_02_01_2023.csv — Error parsing metadata rows: ValueError(AssertionError('26 columns passed, passed data had 40 columns'))
❌ WS500_02_01_2023.csv — Error reading CSV: UnicodeDecodeError('utf-8', b'DATE;TIME;AIR TEMPERATURE (\xb0C);RELATIVE HUMIDITY (%);RELATIVE PRESSURE (hPa);AIR DENSITIY (Kg/m3);WIND SPEED (km/h);WIND DIRECTION (\xb0);ABSOLUTE HUMIDITY (%);ABSOLUTE PRESSURE (hPa);WIND SPEED (m/s)\r\n02/01/2023;00:00:00;19.42;90.23;1006.95;1.19;0.00;0.00;15.04;1006.95;0.00\r\n02/01/2023;00:01:00;19.42;90.20;1006.97;1.19;1.29;52.27;15.03;1006.97;0.36\r\n02/01/2023;00:02:00;19.42;90.37;1007.05;1.19;5.33;71.58;15.06;1007.05;1.48\r\n02/01/2023;00:03:00;19.42;90.14;1007.03;1.19;2.57;130.50;15.02;1007.03;0.71\r\n02/01/2023;00:04:00;19.42;90.02;1007.02;1.19;1.98;37.88;15.00;1007.02;0.55\r\n02/01/2023;00:05:00;19.42;90.00;1007.00;1.19;2.72;149.96;15.00;1007.00

...

C:\Users\syaba\AppData\Local\Temp\ipykernel_22988\4081861802.py:5: DtypeWarning: Columns (2,3,5,6,7,8,2057,2058,2059,2060,2061,2062,2063,2064,2065,2066,2067,2068,2069) have mixed types. Specify dtype option on import or set low_memory=False.
  try: df = pd.read_csv(file_path, sep=';', header=None)


❌ MS711_06_01_2023.csv — Error parsing metadata rows: ValueError(AssertionError('26 columns passed, passed data had 2070 columns'))
[8000/1293711] processed (0.6%) — Skipped: 18 — ETA: 317.4 min
❌ PV_06_01_2023.csv — Error parsing metadata rows: ValueError(AssertionError('26 columns passed, passed data had 40 columns'))
❌ WS500_06_01_2023.csv — Error reading CSV: UnicodeDecodeError('utf-8', b'DATE;TIME;AIR TEMPERATURE (\xb0C);RELATIVE HUMIDITY (%);RELATIVE PRESSURE (hPa);AIR DENSITIY (Kg/m3);WIND SPEED (km/h);WIND DIRECTION (\xb0);ABSOLUTE HUMIDITY (%);ABSOLUTE PRESSURE (hPa);WIND SPEED (m/s)\r\n06/01/2023;00:00:00;18.02;97.66;1005.92;1.19;2.52;224.39;14.98;1005.92;0.70\r\n06/01/2023;00:01:00;18.03;97.64;1005.83;1.19;3.02;211.84;14.99;1005.83;0.84\r\n06/01/2023;00:02:00;18.07;97.65;1005.86;1.19;2.93;212.91;15.02;1005.86;0.81\r\n06/01/2023;00:03:00;18.05;97.53;1005.90;1.19;5.03;205.07;14.99;1005.90;1.40\r\n06/01/2023;00:04:00;18.03;97.55;1005.86;1.19;5.49;210.88;14.98;1005.86;1.52\r\n06

C:\Users\syaba\AppData\Local\Temp\ipykernel_22988\4081861802.py:5: DtypeWarning: Columns (2,3,5,6,7,8,2057,2058,2059,2060,2061,2062,2063,2064,2065,2066,2067,2068,2069) have mixed types. Specify dtype option on import or set low_memory=False.
  try: df = pd.read_csv(file_path, sep=';', header=None)


❌ MS711_07_01_2023.csv — Error parsing metadata rows: ValueError(AssertionError('26 columns passed, passed data had 2070 columns'))
❌ PV_07_01_2023.csv — Error parsing metadata rows: ValueError(AssertionError('26 columns passed, passed data had 40 columns'))
❌ WS500_07_01_2023.csv — Error reading CSV: UnicodeDecodeError('utf-8', b'DATE;TIME;AIR TEMPERATURE (\xb0C);RELATIVE HUMIDITY (%);RELATIVE PRESSURE (hPa);AIR DENSITIY (Kg/m3);WIND SPEED (km/h);WIND DIRECTION (\xb0);ABSOLUTE HUMIDITY (%);ABSOLUTE PRESSURE (hPa);WIND SPEED (m/s)\r\n07/01/2023;00:00:00;18.30;94.70;1005.30;1.19;8.95;146.37;14.77;1005.30;2.48\r\n07/01/2023;00:01:00;18.25;94.98;1005.39;1.19;5.73;143.65;14.77;1005.39;1.59\r\n07/01/2023;00:02:00;18.22;95.03;1005.15;1.19;1.73;92.35;14.76;1005.15;0.48\r\n07/01/2023;00:03:00;18.22;95.06;1005.26;1.19;6.41;111.91;14.76;1005.26;1.78\r\n07/01/2023;00:04:00;18.24;95.12;1005.26;1.19;7.63;145.68;14.78;1005.26;2.12\r\n07/01/2023;00:05:00;18.22;95.27;1005.08;1.19;4.47;72.59;14.79;1005

KeyboardInterrupt: 